# Week 10 - Classification with DecisionTrees

KentB

A **DecisionTree** implementation using **Gini Impurity** as an uncertainty measure.

In [16]:
import pandas as pd
import numpy as np

## Gini Impurity

A measure of Information Gain, aka reduction of uncertainty. 

$1 - \sum_{i=1}^n p_i^2$


## Decision Tree Implementation



**Try to predict wine quality**

In [17]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",sep=';')

In [18]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [19]:
# Define a simple binary feature for quality
df['quality_good'] = np.where(df.quality > 5, 1, 0)

In [20]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_good
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


In [21]:
# Remove target columns from input data
X = df.iloc[:,:-2]
# Target data is in the very last column
target = df.iloc[:,-1]

In [22]:
target

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: quality_good, Length: 1599, dtype: int64

In [23]:
# Useful in the gini calculation?
df.nunique()

fixed acidity            96
volatile acidity        143
citric acid              80
residual sugar           91
chlorides               153
free sulfur dioxide      60
total sulfur dioxide    144
density                 436
pH                       89
sulphates                96
alcohol                  65
quality                   6
quality_good              2
dtype: int64

In [24]:
# Define our target column as a constant
TARGET_NAME = 'quality_good'

In [25]:
  # Test counts of outcomes for some column (here 'quality') - shows correlation w/ target
  df.groupby(target)[['quality']].count().reset_index().rename(columns={'quality':'count'})

,quality_good,count
0,0,744
1,1,855


In [26]:
# Calculate gini for given feature - see formula in notes above
def gini(df, target, feature):
  # Get the count of given features per our target column
  #    Since it's a grouping and we are outputting a count, rename feature as count for clarity
  df_grouped = df.groupby(target)[[feature]].count().reset_index().rename(columns={feature:'count'})
  # Need the full sum to satisfy forumula
  count_sum = df_grouped['count'].sum()
  # Apply gathered stats to gini formula
  return 1 - sum([ (df_grouped[df_grouped[target]==i]['count'].min()/count_sum)**2 for i in df_grouped[target].unique()])

In [27]:
def calc_info_gain(gini_parent, gini_left_child, gini_right_child, weight):
  """
  Calculate information gain

  gini_parent - dataframe filtered to 
  """
  return gini_parent - (gini_left_child * weight + gini_right_child * (1-weight))

In [28]:
def question_and_partition(df_parent, target):
  best_info = 0
  best_question = ()

  if exit_condition(df_parent, target):
    return

  for col in df_parent.columns:
    # Examine each unique value
    values = df_parent[col].unique()
    for val in values:
      # Define left and right sub-trees
      left_part = df_parent[df_parent[col] >= val]
      right_part = df_parent[df_parent[col] < val]
      # Calculate ginis for all 3
      gini_parent = gini(df_parent, target, col)
      gini_left = gini(left_part, target, col)
      gini_right = gini(right_part, target, col)
      # Apply a proportional weight and calculate overall gain
      weight = left_part.shape[0]/df_parent.shape[0]
      info_gain = calc_info_gain(gini_parent, gini_left, gini_right, weight)
      # Keep this value?
      if info_gain > best_info:
        # tuple defines the best question
        best_question = (col, val)
        best_left_part = left_part
        best_right_part = right_part
  return (best_question, best_left_part, best_right_part)

In [29]:
# Test a run
gini(df, TARGET_NAME, 'quality')

0.49759054380845436

In [30]:
# Test a run
#question_and_partition(df, TARGET_NAME)

**Navigate the tree based on feature relevance**

In [31]:
class Node():
  """
  This is a class which enables recursively evaluating the tree
  by retaining a tree node's best value and pointers to its child trees
  """
  def __init__(self, value):
    self.value = value
    self.left = None
    self.right = None
   

In [34]:
def exit_condition(df_parent, target):
  gini_parent = gini(df_parent, target, 'quality')
  if gini_parent == 0:
    return True

In [35]:
def main():
  best_q, best_left, best_right = question_and_partition(df, TARGET_NAME)
  parent = Node(best_question)
  while condition:
      best_left_q, best_left_left, best_left_right = question_and_partition(df_parent, TARGET_NAME)
      left_node = Node()
      parent.left = left_node

      best_right_q, best_right_left, best_right_right = question_and_partition(df_parent, TARGET_NAME)
      right_node = Node()
      parent.right = right_node

